# Getting the Bugs Out
### Troubleshooting Code in Python

In this workshop we'll look at some examples of Python code that's not working as intended or that could be improved, and we'll implement strategies to do so.

### Introduction

#### Discussion Question

What makes good software (vs. bad software)? 

#### Four R's

1. Reliability
2. Robustness
3. Readability
4. Reproducibility

Reliability = Does it perform as expected in the presence of valid inputs?

Robustness = Does it handle invalid inputs gracefully?

Readability = Can others with relevant expertise -- other than the code's author -- understand the code and what it is intended to do?

Reproducibility = Can others use the code on the same inputs and achieve the same (or equivalent) results?


#### Learning objectives
In this workshop, we'll focus on reliability/robustness, looking at examples of code that doesn't perform as expected and revising them make to the code more error-proof. Along the way, we'll talk about best practices that can also make code more readable and reproducible.


### Understanding exceptions in Python

#### Example 1: Analyzing Twitter data

Data from the Twitter API is in [JSON-L](https://jsonlines.org/) format, in which each line of a text file is a valid string in JSON (JavaScript Object Notation). 

We have a file containing a couple thousand Tweets by U.S. Senators. They are all (are should be all) retweets -- meaning, instances where the Twitter user has retweeted someone else's Tweet. 

We want to parse this file and examine which accounts are being retweeted by which Senators?

#### Loading and parsing JSON-L data

Our first step is to import a couple of libraries we'll need for this task.

In [ ]:
import requests
import jsonlines

Now we'll load the Twitter dataset (originally retrieved from GW LAI's [TweetSets](https://tweetsets.library.gwu.edu/) platform, but hosted here on GitHub for convenience).

In [ ]:
data = requests.get('https://raw.githubusercontent.com/gwu-libraries/gwlibraries-workshops/master/python-debugging/sample-retweets.jsonl')

Now let's parse this data with the `jsonlines` module, which lets us iterate over a sequence of JSON objects in a file, converting each to Python data structures.

1. Create a `Reader` object. 
2. Use a [list comprehension](https://realpython.com/list-comprehension-python/) to collect each Tweet in a list called `tweets`.

In [ ]:
reader = jsonlines.Reader(data)
tweets = [r for r in reader]

If you ran the above, you should have gotten an `InvalidLineError`. What do we do now?

Two Approaches
1. Panic

   a. Try as many different modifications of the code as you can think of until something works.
   
   b. Google the error and start copy-pasting code snippets from stackoverflow.
   
   c. Abandon this method of loading the data and try something else entirely.
   

2. Interpet the error to diagnose the exact nature of the problem.

**Note**: Panic is a perfectly _normal_ response. As a seasoned developer, I frequently feel panic when seeing an unexpected error in my code, especially if I'm working on a deadline, or using a new library or language, or doing something that's worked correctly a hundred times before. And sometimes trial-and-error/copy-pasting is the most efficient way to solve a problem.

But unlike some languages, Python is written with care for its exceptions. They are crafted to be readable, informative, and specific. 

#### Anatomy of a Python exception

- Exception = an outcome of a command that the Python interpreter did not anticipate. (Not necessarily an error in the sense of a mistake.)
- Includes one or more "tracebacks." These show the **stack trace**: the nested sequence of Python instructions that the interpreter was trying to execute when it triggered the exception.
- Read each traceback from the top down. Most of the code shown (in this case) is not code we wrote, but code that our code refers to (under the hood). 
- Understand the sequence of tracebacks, using the messages between them. In this case, the first exception in the list is the most immediate: in this case, a `JSONDecodeError`.
- The tracebacks will tell you which Python module/package is causing the exception, which can be helpful in troubleshooting.



Since the exception was caused by our list comprehension, and since it's a JSON-related error, we might reason that something is not working properly with the `reader` object we created. Looking at the [documentation](https://jsonlines.readthedocs.io/en/latest/#jsonlines.Reader) for `jsonlines.Reader`, we see the following:
```
The first argument must be an iterable that yields JSON encoded strings. Usually this will be a readable file-like object, such as an open file or an io.TextIO instance, but it can also be something else as long as it yields strings when iterated over.
```

So the `Reader` object only works on something that _yields strings when iterated over_. The next question is, Does our `data` object meet that requirement? Let's test it.

In [ ]:
[d for d in data]

At first glance these might look like strings. But notice the `b` at the start of every element in this list. That means `byte`, and in Python, a `byte` object is not the same as a `string` object. The lines are also weirdly truncated, which reflects the way that the `requests.get` method works: it actually retrieves packets of bytes from an HTTP connection, which in this case do not correspond to complete Tweets. But JSON and JSON-L are strict standards; an incomplete JSON object cannot be parsed.

If we dig further into the documentation for the [`requests` library](https://docs.python-requests.org/en/latest/), we'll note that the response from `requests.get` has a `text` property as well as a `json()` method. 

Let's see what happens with the latter.

In [ ]:
[d for d in data.json()]

So that didn't work either -- same `JSONDecodeError`. What about the `text` property?

In [ ]:
[d for d in data.text]

At least, we're no longer getting an exception. This is a sequence of strings. But the strings are clearly not JSON objects; they're single characters. 

As it happens, `data.text` is one giant string containing all the data retrieved (all the Tweets). And in Python, iterating over a string yields a sequence of characters.

For the `jsonlines.Reader` to work, we need a sequence of strings, where each string is a **valid** JSON object. Now we know that JSON-L format, which is what Twitter provides, is essentially a sequence of valid JSON objects separated by line breaks. And we can break up a Python string into lines (_i.e._, a sequence of strings) by using the `splitlines` method. 

In [ ]:
[d for d in data.text.splitlines()][0]

So putting it all together, we have this correction to our initial code.

In [ ]:
reader = jsonlines.Reader(data.text.splitlines())
tweets = [r for r in reader]

In [ ]:
tweets[0]

#### Finding accounts retweeted by Senators

Each Tweet that is a retweet has a `retweeted_status` element with information about the original Tweet. So if we look for the `user` property of that (retweeted) Tweet, we could associate it with the `user` property of the retweet, in order to find out which Senators are retweeting which other Twitter accounts. 

One way to do this would be to associate a list of Senatorial accounts with each retweeted account that we find.

The following code is a first pass at doing so.

In [ ]:
# Find the accounts that Senators are retweeting, and record which Senators are retweeting which accounts
# To hold our retweeted user accounts
retweeted_accounts = [] 
# Iterate over all our Tweets
for tweet in tweets:
    # Access the information about the original Tweet (what's being retweeted)
    retweet = tweet['retweeted_status']
    # Account of the original Tweet
    account = retweet['user']
    # Save this and associate with the account of the current (re)Tweet
    retweeted_accounts[account].append(tweet['user']) 

What's the problem here?

A: `retweeted_accounts` is of the wrong type. To associate one data element with another data element, we want a dictionary, not a list.

In [ ]:
retweeted_accounts = {} 
for tweet in tweets:
    retweet = tweet['retweeted_status']
    account = retweet['user']
    retweeted_accounts[account].append(tweet['user']) 

What's going on here?

A: A dictionary key can't be just any Python type. It needs to be "hashable," which in Python means something pretty specific: it must be immutable. Strings, integers, floats, tuples are okay; lists and dictionaries are not.

Let's look at this `retweet['user']` variable.

In [ ]:
retweet['user']

To fix this, we can use the `screen_name` element of the user as the key. While we're at it, we might as well store the same element for the Senators' accounts.

In [ ]:
screen_names = {} 
for tweet in tweets:
    retweet = tweet['retweeted_status']
    screen_name = retweet['user']['screen_name']
    screen_names[screen_name].append(tweet['user']['screen_name']) 

We get a `KeyError` when we try to access a key in a dictionary and the key doesn't exist. 

In this case, that's happening because we try to `append` to an element in the dictionary before that element exists. Python has a special data structure for this kind of situation called a `defaultdict`. We simply specify what **type** of thing the value should be if we try to access a key that's not already in the dictionary. Then Python will take care of creating those elements for us.

In [ ]:
from collections import defaultdict
screen_names = defaultdict(list)
for tweet in tweets:
    retweet = tweet['retweeted_status']
    screen_name = retweet['user']['screen_name']
    screen_names[screen_name].append(tweet['user']['screen_name']) 

Same error, different part of the code. Note that because we haven't seen this one before, yet this line comes earlier in our code, we can infer that the code must be working for at least _some_ of the Tweets in our dataset. (Python always stops at the first exception it encounters, unless you tell it otherwise -- we'll see how, later.)

So there must be Tweets in this dataset that don't have that `retweeted_status` property. To fix this, we can use conditional logic, combined with Python's `dict.get` method, which returns `None` if the supplied key does not exist.

In [ ]:
from collections import defaultdict
screen_names = defaultdict(list)
for tweet in tweets:
    retweet = tweet.get('retweeted_status')
    if retweet:
        screen_name = retweet['user']['screen_name']
        screen_names[screen_name].append(tweet['user']['screen_name']) 

In [ ]:
len(screen_names)

In [ ]:
screen_names

Now we have some working code. At a minimal level, this code is both reliable and robust. It works as expected for valid inputs (Tweets that are retweets) and it doesn't break on invalid inputs (Tweets that are not retweets). Note that if our dataset had not included examples of Tweets that weren't retweets, we wouldn't have necessarily known to account for that situation. The more you test your code on different kinds of inputs, the more robust you will be able to make it.

We might want to go a step further and identify which Tweets are missing the `retweeted_status` element. (If we thought this dataset was supposed to be all retweets, it would be go to know how far off it is from our expectations.)

To do that, we can use a `print` statement in conjunction with the `enumerate` function (which will help us identify the line number of the problematic Tweet in the original dataset).

In [ ]:
from collections import defaultdict
screen_names = defaultdict(list)
for i, tweet in enumerate(tweets):
    retweet = tweet.get('retweeted_status')
    if retweet:
        screen_name = retweet['user']['screen_name'] 
        screen_names[screen_name].append(tweet['user']['screen_name']) 
    else:
        print(f'Tweet in row {i} has no retweet')

#### Catching exceptions

Conditionals are a good way to handle situations where elements may or may not be present in your data. 

There's another approach that actually _uses_ Python's exceptions to build in control flow. Instead of `if...else`, we use `try...except`, which allows us to "catch" exceptions and handle them (instead of having the Python interpreter interrupt our code and return the stack trace). As a logical structure, `try..except` is sort of like an inverted `if...else` block, in that instead of executing something only if a condition is met, the interpreter will "try" to execute some code, and if it fails (on account of an exception), then it will do something else instead.

In [ ]:
from collections import defaultdict
screen_names = defaultdict(list)
for i, tweet in enumerate(tweets):
    try: 
        retweet = tweet['retweeted_status']
        screen_name = retweet['user']['screen_name'] #
        screen_names[screen_name].append(tweet['user']['screen_name']) #
    except KeyError as e:
        print(f'Error in row {i} with missing key {e}')

`try...except` isn't suitable for every situation. Here, however, it buys us a little more generality. Instead of flagging only those rows where the `retweeted_status` element is missing, our new code will catch instances where _any_ of the elements we're interested in are missing from our data. 

In the `except` statement, we specify the kind of exception we're looking for: in this case, a `KeyError`. The `as e` expression will give us a reference to that exception, which in the case of a `KeyError`, we can use to identify the missing key. 

Note that any other kind of exception (other than `KeyError`) that might occur inside the `try` block would **not** get caught and so would interrupt execution of the code. But that's actually a good thing. You don't want to catch **every** error in your code; some you **do** want to allow to "bubble up" and interrupt the execution. Otherwise, you may end up ignoring errors that can distort your results. 

As a general rule of thumb: a **noisy** exception is better than a **silent** error.

### Best practices

#### Encapsulation

In addition to catching exceptions, there are other patterns conducive to bug-free code. One very common one is to use functions generously to encapsulate discrete behaviors in your code. 

Functions allow us to avoid repeating ourselves, which in turn reduces the chance of syntax errors and the like. Functions also make code easier to test, debug, and reason about.

Let's say we want to load another dataset of Tweets, also in JSON-L. This dataset contains a mix of different kinds of Tweets (including retweets and quotes as well as original Tweets).

We could re-run the cell we use above or copy and paste the code into a new cell. But if this is something we might need to do regularly, it's useful to define a new function to contain this logic.

In [ ]:
def load_tweets(url_of_tweets):
    data = requests.get(url_of_tweets)
    reader = jsonlines.Reader(data.text.splitlines())
    tweets = [r for r in reader]
    return tweets

Functions consist of the following:
- a `def` statement followed by a unique name (try to avoid names used by other Python functions) and parentheses
- optionally, a list of parameters inside the parentheses. Paremeters with a default value (which take the form `parameter=default_value`) are optional; if not provided, they receive the default value. Paremeters without a default value are required. If they are missing from the function call, Python raises an exception.
- a `return` statement. (Otherwise, the function generally has no effect.)

We can use our function to load a new dataset.

In [ ]:
tweets_2 = load_tweets('https://raw.githubusercontent.com/gwu-libraries/gwlibraries-workshops/master/python-debugging/tweets-shuffled.jsonl')

Functions can also make complicated logic easier to read by breaking it into smaller units. Let's say we want to extract from each Tweet the account screen name, the number of times it was retweeted, and the text of the Tweet, and we want to include retweeted and quoted Tweets, too.

#### Writing DRY code

In [ ]:
tweet_metadata = []
for tweet in tweets_2:
    rt = tweet.get('retweeted_status')
    if rt:
        rt_screen_name = tweet['user']['screen_name']
        rt_text = tweet['full_text']
        rt_num_rt = tweet['retweet_count']
        tweet_metadata.append({'screen_name': rt_screen_name,
                               'text': rt_text,
                               'num_retweets': rt_num_rt})
    else:
        if tweet.get('quoted_status'):
            qt_screen_name = tweet['user']['screen_name']
            qt_text = tweet['full_text']
            qt_num_rt = tweet['retweet_count']
            tweet_metadata.append({'screen_name': qt_screen_name,
                                   'text': qt_text,
                                   'num_retweets': qt_num_rt})
    screen_name = tweet['user']['screen_name']
    text = tweet['full_text']
    num_rt = tweet['retweet_count']
    tweet_metadata.append({'screen_name': screen_name,
           'text': text,
           'num_retweets': num_rt})

What do you notice about the code above?

A: It's both hard to read and redundant. We've got a lot of variables that are rather similar, which in itself is a sign that we could use a function to avoid repeating ourselves. 

Instead of packing all that logic in our for loop, let's make a function to handle the extraction of the metadata. We will also include a [docstring](https://www.python.org/dev/peps/pep-0257/) to remind our future selves what this function does.

In [ ]:
def extract_metadata(tweet):
    '''
    Extracts the screen name, number of retweets, and text of a tweet. 
    Returns a dictionary containing those elements.
    :param tweet: a Python dictionary corresponding to a Tweet document
    '''
    screen_name = tweet['user']['screen_name']
    text = tweet['full_text']
    num_rt = tweet['retweet_count']
    return {'screen_name': screen_name,
           'text': text,
           'num_retweets': num_rt}

In [ ]:
tweet_metadata = []
for tweet in tweets_2:
    # We can use the fact that dict.get returns None if the key isn't found to condense the two branches of our conditional into a single Boolean expression
    rtq = tweet.get('retweeted_status') or tweet.get('quoted_status')
    if rtq:
        tweet_metadata.append(extract_metadata(rtq))
    tweet_metadata.append(extract_metadata(tweet))

In [ ]:
len(tweet_metadata)

#### Working with third-party libraries

#### Example 2: Residential household energy consumption in Arlington, VA by hour and day, 2014

Debugging code that uses third-party Python libraries presents its own challenges. The following example highlights some pitfalls with datetime values in the pandas library and illustrates how built-in library methods can simplify and optimize your code.

The data source is originall from the [Open Energy Data Initiative](https://data.openei.org/submissions/153), but I've modified it for this lesson. It has one row for each day/hour in the year for 2014, and columns corresponding to the energy usage for various types of utilities and appliances.

In [ ]:
import pandas as pd

In [ ]:
energy_df = pd.read_csv('https://raw.githubusercontent.com/gwu-libraries/gwlibraries-workshops/master/python-debugging/energy-consumption-arlington-2014.csv')

In [ ]:
energy_df

#### Troubleshooting Date/Time problems

pandas has a handy `to_datetime` method that can convert a string to a Python datetime value. That will be useful if, say, we want to plot this data as a time series. But running it on the `Date/Time` column throws an exception.

In [ ]:
pd.to_datetime(energy_df['Date/Time'])

In this case, the bottom-most traceback is the most useful. Exceptions in pandas are not always transparent because some of the underlying methods are written in C, not Python, so the code doesn't show up in the stack trace. But if we compare the timestamp shown in the `OutOfBoundsDatetime` exception with the first value in our column, we notice that something is going on with the parsing of the date. 

Our `Date/Time` values are missing the year. We know this dataset is for 2014, but pandas has no way of knowing that, and without a year, it can't create a valid date (so it seems to be trying to make it valid by appending a `1`, and then it thinks that's out of bounds. 

To deal with this, we need to add the year to the `Date/Time` column. There are a few ways of handing this.

A first approach might be to use a 'for' loop.

In [ ]:
for dt in energy_df['Date/Time']:
    day, hour = dt.split()
    dt = day + '/2014' + ' ' + hour

In [ ]:
energy_df['Date/Time']

Any guesses as to why this didn't work?

A: Iterating over a column of a pandas DataFrame uses a *copy* of the data in the DataFrame. So modifying the values in that copy has no effect on the original.

It's better to use pandas' built-in syntax for string operations. Here we create a new DataFrame that consists of the day and hour parts of the string, and then we add the year onto the day before concatenating both into a new string in a new column of our original DataFrame.

And the great thing about pandas is that we can do all this *without a `for` loop!*

In [ ]:
dt_df = energy_df['Date/Time'].str.strip().str.split(' ', expand=True)

In [ ]:
dt_df[0] = dt_df[0] + '/2014'

In [ ]:
energy_df['timestamp'] = dt_df[0] + ' ' + dt_df[1]

Now we should be able to convert our `timestamp` column with `pandas.to_datetime`.

In [ ]:
energy_df['Date/Time'] = pd.to_datetime(energy_df.timestamp)

#### When `for` loops are superfluous

Finally, let's say we wanted to plot the usage of these various utilities over the course of the year. Hourly data often proves rather noisy when plotted, which can make it hard to discern patterns. 

Plotting just one type of utility makes for a very noisy chart, so it will be hard to compare different utilities/types of consumption.

In [ ]:
energy_df.plot(x='Date/Time', y='Electricity:Facility [kW](Hourly)')

In these cases, the [rolling mean](https://en.wikipedia.org/wiki/Moving_average) can be a useful technique. By smoothing values over a particular period of time, it can reveal trends that emerge at different time scales. It lets you "zoom out" on the data, so to speak. 

Since the data is hourly, we might want to calculate the rolling mean over a 24-hour period, which would give us a better impression of the trends from day to day. 

Our `for` loop approach to the rolling mean uses pandas indexing and slicing to average over the last 24 hours of data, hour by hour in our dataset. 

In [ ]:
mean_kw = []
for i in range(23, len(energy_df)):
    first_idx = i - 23
    period = energy_df.iloc[first_idx:i+1]['Electricity:Facility [kW](Hourly)']
    mean_kw.append(period.mean())

Now we have a list containing the rolling hourly average for this particular measure. But in the process, we've lost our time series axis, which will make it harder to plot.

In [ ]:
mean_kw

Fortunately, pandas has us covered with another built-in method. We simply specify a period length as an argument to the `rolling` method, then we call one of pandas' aggregate methods on the result of that `rolling` method -- in this case, `mean()`. 

In [ ]:
energy_df.rolling(24).mean()

We'd like to calculate a rolling mean for all the metrics in our DataFrame. But pandas is now complaining about our newly converted `Date/Time` column -- and we need that column for plotting...

Not to worry. Here's where the pandas index comes in especially handy.

We can make the `Date/Time` column into the index of our DataFrame. Then the `rolling` method won't try to compute with its values, but we can still use those values for plotting. While we're at it, we'll get rid of the intermediate `timestamp` column, since we have no more need of it.

In [ ]:
energy_df = energy_df.drop('timestamp', axis=1)
energy_df = energy_df.set_index('Date/Time')

In [ ]:
rolling = energy_df.rolling(24).mean()

Now we can see some interesting trend lines for all the different types of utilities/appliances in our dataset.

In [ ]:
rolling.plot(figsize=(12,12))